In [1]:
import os

from dotenv import load_dotenv

In [2]:
from langchain_community.vectorstores import FAISS
from utils import load_embeddings, initialize_llm
from prompt import contextualize_q_system_prompt, system_prompt


from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [3]:
load_dotenv()

True

In [4]:
os.environ['GOOGLE_API_KEY'] = os.environ.get('GOOGLE_API_KEY')
os.environ['GROQ_API_KEY'] = os.environ.get('GROQ_API_KEY')

In [5]:
embeddings = load_embeddings(embd_type='GoogleGenAI')

## Load the persisted database from disk

In [ ]:
# load the VectorDB
db_directory_path="Gale_Encyclopedia_of_Med_db"
docsearch = FAISS.load_local(db_directory_path, embeddings, allow_dangerous_deserialization=True)

## Make a Retriever (Similarity Search)

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":5}) 

In [ ]:
llm = initialize_llm(temp=0.6)